<a href="https://colab.research.google.com/github/shyamjith94/Machine-Learning-Regression/blob/main/MultipleLinearRegressionGradientDescent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
%%capture
!pip install turicreate

In [43]:
import turicreate as tc
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from math import log, sqrt

In [44]:
data = tc.SFrame.read_csv("/content/drive/My Drive/Machine Learning: Regression/Data/2/c_kc_house_data.csv/kc_house_data.csv")

Finished parsing file /content/drive/My Drive/Machine Learning: Regression/Data/2/c_kc_house_data.csv/kc_house_data.csv

Parsing completed. Parsed 100 lines in 0.150647 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,float,int,float,int,int,float,int,int,int,int,int,int,int,int,int,float,float,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /content/drive/My Drive/Machine Learning: Regression/Data/2/c_kc_house_data.csv/kc_house_data.csv

Parsing completed. Parsed 21613 lines in 0.100655 secs.

In [45]:
data.head()

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view
7129300520,20141013T000000,221900.0,3,1.0,1180,5650,1.0,0,0
6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0
5631500400,20150225T000000,180000.0,2,1.0,770,10000,1.0,0,0
2487200875,20141209T000000,604000.0,4,3.0,1960,5000,1.0,0,0
1954400510,20150218T000000,510000.0,3,2.0,1680,8080,1.0,0,0
7237550310,20140512T000000,1225000.0,4,4.5,5420,101930,1.0,0,0
1321400060,20140627T000000,257500.0,3,2.25,1715,6819,2.0,0,0
2008000270,20150115T000000,291850.0,3,1.5,1060,9711,1.0,0,0
2414600126,20150415T000000,229500.0,3,1.0,1780,7470,1.0,0,0
3793500160,20150312T000000,323000.0,3,2.5,1890,6560,2.0,0,0


In [46]:
def convert_to_numpy(frame: tc.SFrame, features: list, output:str):
    """ Coverting Sframe to numpy array and return it
        frame   - Sframe 
        feature - list of feature variable
        output  - its output variable of model our case price"""
    # add constant column to frame
    frame["constant"] = 1
    # combine two list
    features = ["constant"] + features
    # cutting data frame based on feature list
    feature_frame = frame[features]
    # convert to numpy
    feature_metrix = feature_frame.to_numpy()
    # cutting frame based on output array
    output_array = frame[output]
    output_metrix = output_array.to_numpy()
    return (feature_metrix, output_metrix)

In [47]:
sample_fetaure, sample_ouput = convert_to_numpy(data, ["sqft_living"], "price")
print("sample_fetaure")
print(sample_fetaure[:5])
print("sample_ouput")
print(sample_ouput[:5])

sample_fetaure
[[   1 1180]
 [   1 2570]
 [   1  770]
 [   1 1960]
 [   1 1680]]
sample_ouput
[221900. 538000. 180000. 604000. 510000.]


In [48]:
my_weights = np.array([1., 1.]) # the example weights
my_features = sample_fetaure[0,] # we'll use the first data point
predicted_value = np.dot(my_features, my_weights)
predicted_value

1181.0

In [49]:
def prediction(feature: np.array, weight: np.array):
    """ Making Prediction using and Np.Dot fucntion
        feature is numpy array return by convert numpy fun
        weight is custom weight is numpy array"""
    prediction = np.dot(feature, weight)
    return prediction


In [50]:
sample_prediction = prediction(sample_fetaure, my_weights)
sample_prediction

array([1181., 2571.,  771., ..., 1021., 1601., 1021.])

**Computing derivative**

In [51]:
def feature_derivative(error, feature):
    """ calculating feature derivative using error and feature
        feature is numpy array and error is predicton substract
        from output(price)"""
    # compute twice of np dot product value
    derivative = 2 * np.dot(error, feature)
    return derivative

In [52]:
sample_fetaure, sample_ouput = convert_to_numpy(data, ["sqft_living"], "price")
new_weight = np.array([0., 0.])
# making prediction
new_prediction = prediction(sample_fetaure, new_weight)
# calculating errror 
new_feature = sample_fetaure[:,0]
error = new_prediction - sample_ouput
# finding derivative
derivative = feature_derivative(error=error, feature=new_feature)
# checking same as derivative of sample ouput
sum_squared_sample_output = np.sum(sample_ouput)*2

# printing values
print("New Feaure Is ")
print(new_feature,"\n")
print("New Error Is")
print(error, "\n")
print("New Derivative Is                        :-\t", derivative)
print("New Derivative Of SampleOutput Squared   :-\t", -sum_squared_sample_output)

New Feaure Is 
[1 1 1 ... 1 1 1] 

New Error Is
[-221900. -538000. -180000. ... -402101. -400000. -325000.] 

New Derivative Is                        :-	 -23345850016.0
New Derivative Of SampleOutput Squared   :-	 -23345850016.0


**Gradient Descent**


In [53]:
def gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    """ Computing gradient descent, algorithm will check the "converge" of 
        based of "tolerance" calculate the gradient sum of square sqrt it
        to get gradient magnitude feature matrix array of feature outout 
        will we price of house"""
    
    converged = False 
    weights = np.array(initial_weights) # make sure it's a numpy array
    while not converged:
        # compute the predictions based on feature_matrix and weights using your predict_output() function
        predictions = prediction(feature_matrix, weights)
        
        # compute the errors as predictions - output
        errors = predictions - output

        gradient_sum_squares = 0 # initialize the gradient sum of squares
        # while we haven't reached the tolerance yet, update each feature's weight
        for i in range(len(weights)): # loop over each weight
            # Recall that feature_matrix[:, i] is the feature column associated with weights[i]
            # compute the derivative for weight[i]:
            derivative = feature_derivative(errors, feature_matrix[:, i])

            # add the squared value of the derivative to the gradient sum of squares (for assessing convergence)
            squared_derivative = derivative * derivative 
            gradient_sum_squares = gradient_sum_squares + squared_derivative
            print("Gradient Sum Square \t And Gradient Magnitude And \t Tolerance")
            print(gradient_sum_squares, "\t", sqrt(gradient_sum_squares), "\t\t", tolerance)
            
            # subtract the step size times the derivative from the current weight
            weights[i] = weights[i] - step_size * derivative
            
        # compute the square-root of the gradient sum of squares to get the gradient magnitude:
        gradient_magnitude = sqrt(gradient_sum_squares)
        # print(gradient_magnitude, tolerance)
        if gradient_magnitude < tolerance:
            converged = True
    return(weights)
    

In [54]:
train_data,test_data = data.random_split(.8,seed=0)

In [55]:
# test out the gradient descent
simple_features = ['sqft_living']
my_output = 'price'
(simple_feature_matrix, output) = convert_to_numpy(train_data, simple_features, my_output)
initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7

In [56]:
weights = gradient_descent(simple_feature_matrix, output, initial_weights, step_size, tolerance)
print("\nThe value of the weight for sqft_living:", weights)

Gradient Sum Square 	 And Gradient Magnitude And 	 Tolerance
4.126779536001204e+20 	 20314476454.0 		 25000000.0
Gradient Sum Square 	 And Gradient Magnitude And 	 Tolerance
2.555457264704116e+27 	 50551530784973.43 		 25000000.0
Gradient Sum Square 	 And Gradient Magnitude And 	 Tolerance
2.7838186677693055e+19 	 5276190545.999363 		 25000000.0
Gradient Sum Square 	 And Gradient Magnitude And 	 Tolerance
1.7232997044781152e+26 	 13127451026296.443 		 25000000.0
Gradient Sum Square 	 And Gradient Magnitude And 	 Tolerance
1.875010002966224e+18 	 1369310046.3248723 		 25000000.0
Gradient Sum Square 	 And Gradient Magnitude And 	 Tolerance
1.1621254295552885e+25 	 3408996083241.0596 		 25000000.0
Gradient Sum Square 	 And Gradient Magnitude And 	 Tolerance
1.2703786753327899e+17 	 356423719.0946739 		 25000000.0
Gradient Sum Square 	 And Gradient Magnitude And 	 Tolerance
7.836916065790874e+23 	 885263580285.0399 		 25000000.0
Gradient Sum Square 	 And Gradient Magnitude And 	 Tolerance


In [57]:
(test_simple_feature_matrix, test_output) = convert_to_numpy(test_data, simple_features, my_output)

In [58]:
prediction_test = prediction(test_simple_feature_matrix, weights)
prediction_test

array([356134.443255  , 784640.86440132, 435069.83662406, ...,
       663418.65315598, 604217.10812919, 240550.47439317])

In [59]:
# actual price of first house and predicted price
print("predicted price of first house:-\t",prediction_test[0], "\nactual price of first house:-\t\t",
test_data['price'][0])

predicted price of first house:-	 356134.4432550024 
actual price of first house:-		 310000.0


**Compute RSS Prediction**

In [60]:
def get_residual_sum_of_squares(predictions, output):
    residuals = output - predictions
    # Then square and add them up
    squared_residuals = residuals*residuals
    residual = squared_residuals.sum()
    return (residual)

In [61]:
residual_sum = get_residual_sum_of_squares(prediction_test, test_output)
residual_sum

275400044902128.3

**Running Multile Regression**

In [62]:
model_features = ['sqft_living', 'sqft_living15']
my_output = 'price'
(feature_matrix, output) = convert_to_numpy(train_data, features=model_features, output=my_output)
initial_weights = np.array([-100000., 1., 1.])
step_size = 4e-12
tolerance = 1e9

In [63]:
model_weights = gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance)
print("\nThe value of the weight for multiple feature:", model_weights)

Gradient Sum Square 	 And Gradient Magnitude And 	 Tolerance
4.878855478601407e+20 	 22088131380.0 		 1000000000.0
Gradient Sum Square 	 And Gradient Magnitude And 	 Tolerance
2.9402584584764944e+27 	 54224150140656.836 		 1000000000.0
Gradient Sum Square 	 And Gradient Magnitude And 	 Tolerance
5.339520188136633e+27 	 73072020556001.0 		 1000000000.0
Gradient Sum Square 	 And Gradient Magnitude And 	 Tolerance
5.078298899920748e+19 	 7126218422.08106 		 1000000000.0
Gradient Sum Square 	 And Gradient Magnitude And 	 Tolerance
2.6266706462149208e+26 	 16207006652108.59 		 1000000000.0
Gradient Sum Square 	 And Gradient Magnitude And 	 Tolerance
5.140749490426569e+26 	 22673220967534.74 		 1000000000.0
Gradient Sum Square 	 And Gradient Magnitude And 	 Tolerance
3.768924342170467e+18 	 1941371768.150157 		 1000000000.0
Gradient Sum Square 	 And Gradient Magnitude And 	 Tolerance
3.16218906012336e+25 	 5623334473533.795 		 1000000000.0
Gradient Sum Square 	 And Gradient Magnitude And 	 T

In [64]:
(test_model_feature_matrix, test_output) = convert_to_numpy(test_data, model_features, my_output)

In [65]:
predictions_test_model_2 = prediction(test_model_feature_matrix, model_weights)
predictions_test_model_2

array([366651.41162949, 762662.39850726, 386312.09557541, ...,
       682087.39916306, 585579.27901327, 216559.20391786])

In [66]:
# actual price of first house and predicted price
print("predicted price of first house:-\t",predictions_test_model_2[0], "\nactual price of first house:-\t\t",
test_data['price'][0])

predicted price of first house:-	 366651.4116294939 
actual price of first house:-		 310000.0


In [68]:
get_residual_sum_of_squares(predictions_test_model_2, test_output)

270263443629803.56

In [70]:
# The multiple regression model has lower RSS than Simple model
np.min(np.array([275400044902128, 270263443629803]))

270263443629803